In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [12]:
df = pd.DataFrame(pd.read_csv("dt.csv"))
df.head()

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1996.72,2029.99,-1.638924,2005.50,2005.56,1950.00,2063.73,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,267098.00,269649.00,-0.946045,267124.00,267267.00,267124.00,267267.00,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36
2,2017-05-31,2017-06-01 00:00:00,korbit,btc_krw,3003500.00,3140000.00,-4.347134,3003500.00,3004000.00,3002000.00,3209500.00,6109.752872,2017-05-31 14:59:36,2017-05-31 14:59:36
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36


In [13]:
df.keys()

Index(['date_id', 'datetime_id', 'market', 'rpt_key', 'last', 'diff_24h',
       'diff_per_24h', 'bid', 'ask', 'low', 'high', 'volume', 'created_at',
       'updated_at'],
      dtype='object')

In [14]:
df = df.rename(columns={"date_id": "Date", "datetime_id": "Time", "market": "Market", "rpt_key": "RPT Key", "last": "Last", "diff_24h": "Diff (24H)",
          "diff_per_24h": "Diff/24H", "bid": "BID", "ask": "Ask", "low": "Low", "high": "High", "volume": "Volume", "created_at": "Created At", "updated_at": "Update At"})
df.head()

,Date,Time,Market,RPT Key,Last,Diff (24H),Diff/24H,BID,Ask,Low,High,Volume,Created At,Update At
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1996.72,2029.99,-1.638924,2005.50,2005.56,1950.00,2063.73,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,267098.00,269649.00,-0.946045,267124.00,267267.00,267124.00,267267.00,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36
2,2017-05-31,2017-06-01 00:00:00,korbit,btc_krw,3003500.00,3140000.00,-4.347134,3003500.00,3004000.00,3002000.00,3209500.00,6109.752872,2017-05-31 14:59:36,2017-05-31 14:59:36
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36


In [15]:
df2 = df.loc[(df['RPT Key'] == 'eth_krw')]
df2.head()

,Date,Time,Market,RPT Key,Last,Diff (24H),Diff/24H,BID,Ask,Low,High,Volume,Created At,Update At
7,2017-05-31,2017-06-01 00:00:00,korbit,eth_krw,311800.0,274500.0,13.588342,311800.0,311950.0,272500.0,336000.0,327416.949269,2017-05-31 14:59:36,2017-05-31 14:59:36
11,2017-05-31,2017-06-01 00:01:00,korbit,eth_krw,311900.0,272500.0,14.458716,311900.0,311950.0,273000.0,336000.0,326531.242837,2017-05-31 15:00:36,2017-05-31 15:00:36
27,2017-06-01,2017-06-01 00:02:00,korbit,eth_krw,311950.0,275000.0,13.436364,311650.0,311950.0,274000.0,336000.0,326480.640045,2017-05-31 15:01:36,2017-05-31 15:01:36
37,2017-06-01,2017-06-01 00:03:00,korbit,eth_krw,311700.0,274500.0,13.551913,311700.0,311800.0,274000.0,336000.0,326347.137089,2017-05-31 15:02:36,2017-05-31 15:02:36
47,2017-06-01,2017-06-01 00:04:00,korbit,eth_krw,311700.0,280200.0,11.241970,311700.0,311950.0,274000.0,336000.0,326195.186608,2017-05-31 15:03:37,2017-05-31 15:03:37


In [21]:
df2 = df2.reset_index(drop=True)
df2['Date'] = pd.to_datetime(df2['Time'])
df2 = df2.loc[df2['Date'] > pd.to_datetime('2017-06-28 00:00:00')]
LastPrice = df2[['Last']].values.astype('float32')
LastPrice

array([[286550.],
       [286550.],
       [286550.],
       ...,
       [239000.],
       [239100.],
       [239100.]], dtype=float32)

In [22]:
scaledPrice = MinMaxScaler(feature_range=(0, 1))
scaledPrice = scaledPrice.fit_transform(LastPrice)
scaledPrice

array([[0.36366272],
       [0.36366272],
       [0.36366272],
       ...,
       [0.08720934],
       [0.08779073],
       [0.08779073]], dtype=float32)